In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 3

@author: yaning
"""

import numpy as np
import pickle
from tensorflow.keras import datasets

path = "/home/yaning/Documents/"
np.set_printoptions(threshold=np.inf)

2025-06-03 15:01:56.597995: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-03 15:01:56.600544: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-03 15:01:56.606881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748955716.617964 2465388 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748955716.621232 2465388 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748955716.630685 2465388 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Import the features

In [2]:
with open(path + "gabor_data_10.pkl", "rb") as f:
    featured_dataset = pickle.load(f)

In [11]:
featured_dataset.shape

(60000, 10, 10, 6)

normalise them independently

In [12]:
data_norm = np.zeros_like(featured_dataset)
for i in range(featured_dataset.shape[0]):
    img = featured_dataset[i]
    img_min = img.min()
    img_max = img.max()
    if img_max > img_min:
        data_norm[i] = (img - img_min) / (img_max - img_min)
    else:
        data_norm[i] = 0

Generate different firing sequences for different frequencies 

In [51]:
# form an shape (5000, 3) array with 5000 time points and 3 input neurons
# every fire is 45-50 cycles, 
pointCount = 6000
start_tstep = 500
one_cycle = np.concatenate((np.ones(45, dtype=bool), np.zeros(189, dtype=bool)))
firing_duration = 45+189
# 23*(45+189) = 5382
total_time = 5382


non_fire_array = np.zeros(pointCount, dtype=bool)

In [48]:
test = [1,2,3]
test[1:2] = [3]

In [49]:
test

[1, 3, 3]

In [44]:
starting_seq = []
for num_spikes in range(23):
    num_spikes += 1
    segment_length = total_time / num_spikes

    starts = []
    for i in range(num_spikes):
        # Place each firing block in the center of its segment
        center = int((i + 0.5) * segment_length)

        start = max(0, center - firing_duration // 2)

        starts.append(start + start_tstep)
    starting_seq.append(starts)


In [45]:
for i in starting_seq:
    print(i)

[3074]
[1728, 4419]
[1280, 3074, 4868]
[1055, 2401, 3746, 5092]
[921, 1997, 3074, 4150, 5226]
[831, 1728, 2625, 3522, 4419, 5316]
[767, 1536, 2305, 3074, 3842, 4611, 5380]
[719, 1392, 2064, 2737, 3410, 4083, 4755, 5428]
[682, 1280, 1878, 2476, 3074, 3672, 4270, 4868, 5466]
[652, 1190, 1728, 2266, 2804, 3343, 3881, 4419, 4957, 5495]
[627, 1116, 1606, 2095, 2584, 3074, 3563, 4052, 4541, 5031, 5520]
[607, 1055, 1504, 1952, 2401, 2849, 3298, 3746, 4195, 4643, 5092, 5540]
[590, 1004, 1418, 1832, 2246, 2660, 3074, 3488, 3902, 4316, 4730, 5144, 5558]
[575, 959, 1344, 1728, 2112, 2497, 2881, 3266, 3650, 4035, 4419, 4803, 5188, 5572]
[562, 921, 1280, 1638, 1997, 2356, 2715, 3074, 3432, 3791, 4150, 4509, 4868, 5226, 5585]
[551, 887, 1223, 1560, 1896, 2233, 2569, 2905, 3242, 3578, 3914, 4251, 4587, 4924, 5260, 5596]
[541, 857, 1174, 1491, 1807, 2124, 2440, 2757, 3074, 3390, 3707, 4023, 4340, 4656, 4973, 5290, 5606]
[532, 831, 1130, 1429, 1728, 2027, 2326, 2625, 2924, 3223, 3522, 3821, 4120, 4419,

In [58]:
# fire sequence by frequency from low to high
fire_seq = []
for i in starting_seq:
    fire_array = non_fire_array.copy()
    for j in i:
        fire_array[j:j+firing_duration] = one_cycle
    fire_seq.append(fire_array)    

In [59]:
fire_seq = np.array(fire_seq)

In [60]:
fire_seq.shape

(23, 6000)

In [ ]:
fire_seq[0]

In [63]:
with open(path + "Spiking_NN/datasets/fire_seq_in_frequency.pkl", "wb") as f:
    pickle.dump(fire_seq, f)

Use the firing sequence with different frequencies to generate firing array for the dataset  

In [64]:
data_norm.shape

(60000, 10, 10, 6)

In [6]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [7]:
x_train.shape

(60000, 28, 28, 1)